# Scraping Data

Scraping  Data from LinkedIn   for 2 raisons :
 - Updating our databse 
 - to scrap informations about companies  to see if the companw is good for an experience or not  for the condidate 

## 1- Updating Database 

#### #importation des bibleotheques

In [1]:
from selenium import webdriver
from time import sleep
from json import dump
from pymongo import MongoClient
import json
import pandas as pd 
import numpy as np
from pandas.io.json import json_normalize

#### #Data importation

In [2]:
df=[]
for line in open("weevio.json",encoding='utf-8'):
    df.append(json.loads(line))

#### #transformation from json to dataframe

In [3]:
df=pd.DataFrame.from_records(df)

### =>  we used in our data scraping selenium 


In [4]:
client = MongoClient()
link= client.get_database('admin').get_collection('BaseWevioo')
data = list(link.find())

In [5]:
#creation du liste de notre données scrappy 
upt = [{'url':item['url'], 'recomendations':[], "certifs":[], "experience":[]} for item in data ]
company_urls = []

In [7]:
# specifies the path to the chromedriver.exe
driver = webdriver.Chrome('C:/Users/Marwa/Desktop/chromedriver.exe')

# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.linkedin.com/uas/login')

# locate email form by_class_name
username = driver.find_element_by_css_selector('#username')
# send_keys() to simulate key strokes
username.send_keys('iheb.mensi@hotmail.fr')

# locate password form by_class_name
password = driver.find_element_by_css_selector('#password')

# send_keys() to simulate key strokes
password.send_keys('987456321marwa')

# locate submit button by_class_name
username.submit()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 79


In [16]:
# we scraping 10 profils only to show you the execution ,but we did more than that 
#looping urls and scrapping data
for i in range(0,10):
    driver.get(upt[i]['url'])

    #recommendation scrapping
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(1)
    try:
        recom_section = driver.find_element_by_css_selector('.pv-recommendations-section')
        buttons = recom_section.find_elements_by_css_selector('.no-wrap .ember-view')
        driver.execute_script("window.scrollTo(0, "+str(buttons[0].location['y']-400)+");")
        sleep(0.5)
        buttons[0].click()
        try:
            plus_rec = recom_section.find_element_by_css_selector('.pv-recommendations-section .artdeco-button--fluid')
            driver.execute_script("window.scrollTo(0, "+str(plus_rec.location['y']-400)+");")
            plus_rec.click()
        except:
            pass
        learn_more = recom_section.find_elements_by_css_selector('.pv-recommendations-section .lt-line-clamp__more')
        for element in learn_more:
            try :
                driver.execute_script("window.scrollTo(0, "+str(element.location['y']-300)+");")
                element.click()
            except:
                pass
        texts = recom_section.find_elements_by_css_selector('.pv-recommendation-entity__text .ember-view')
        upt[i]['recomendations'] = list(map(lambda x: x.text, texts))
    except:
        pass    

    #certifs scrapping
    try :
        try :
            button_cert = driver.find_element_by_css_selector('#certifications-section button.pv-profile-section__see-more-inline')
            driver.execute_script("window.scrollTo(0, "+str(button_cert.location['y']-300)+");")
            button_cert.click()
        except :
            pass
        certifs_company = list(map(lambda x: (x.text.split("\n")[0],x.text.split("\n")[2]) ,driver.find_elements_by_css_selector("#certifications-section div.pv-certifications__summary-info") )) 
        upt[i]['certifs']=certifs_company
    except :
        pass
    #jobs scrapping
    try :
        try :
            button = driver.find_element_by_css_selector('#experience-section button.pv-profile-section__see-more-inline ')
            driver.execute_script("window.scrollTo(0, "+str(button.location['y']-300)+");")
            button.click()
        except :
            pass 
        text = recom_section.find_elements_by_css_selector('.pv-experience-section-entity__text .ember-view')
        #experience.append( list(map(lambda x: x.text, text)),driver.find_elements_by_css_selector('.pv-experience-section-entity__text .ember-view
        company_urls.append( list(map(lambda x: x.get_attribute('href'), driver.find_elements_by_css_selector('#experience-section a[data-control-name="background_details_company"]'))))
    except:
        pass 
    print(upt[i])

{'url': 'https://www.linkedin.com/in/imen-hammi-3102919a', 'recomendations': [], 'certifs': [], 'experience': []}
{'url': 'https://www.linkedin.com/in/zied-brahmi-67787b48', 'recomendations': [], 'certifs': [], 'experience': []}
{'url': 'https://www.linkedin.com/in/ines-mahjoub-353ba130', 'recomendations': [], 'certifs': [], 'experience': []}
{'url': 'https://www.linkedin.com/in/omar-ghazouani-4a6890151', 'recomendations': [], 'certifs': [], 'experience': []}
{'url': 'https://www.linkedin.com/in/kammoun-walid-34b73171', 'recomendations': [], 'certifs': [], 'experience': []}
{'url': 'https://www.linkedin.com/in/fdebbabitb', 'recomendations': [], 'certifs': [], 'experience': []}
{'url': 'https://www.linkedin.com/in/hanen-ben-ali-82617591', 'recomendations': [], 'certifs': [], 'experience': []}
{'url': 'https://www.linkedin.com/in/ali-daboussi-b5a5979b', 'recomendations': [], 'certifs': [], 'experience': []}
{'url': 'https://www.linkedin.com/in/anisthameur', 'recomendations': [], 'certifs

## -2 Scraping informations about companies 

In [15]:
df=[]
for line in open("weevio.json",encoding='utf-8'):
    df.append(json.loads(line))
df1=pd.DataFrame.from_records(df)
df1.replace('', 'None', regex=True)
experiences = pd.json_normalize(df1['experiences']) 

In [16]:
experiences['jobs'][0][1]['company']

"Atelier d'architecture Chaker ZAYATI"

In [17]:
# Try to scrap all the companies even if there is no link for a company 
# we tryed to add the name of a company with the link of linkedIn
list=[]
listall =[0 for i in range(10000)]
for i in range(0,10000):
    c=len(experiences['jobs'][i])
    
    for j in range(0,c-1):
        if experiences['jobs'][i][j]['company'] != None:
            
            list.append('https://www.linkedin.com/company/'+experiences['jobs'][i][j]['company'].replace(" ","-")+'/')
    #listall[i]= list.copy()
    #list.clear()
#listall[1][1]

In [18]:
# Create Our list for url

listurl = []
for i in list:
    if i not in listurl:
          listurl.append(i)

In [19]:
len(listurl)

15303

In [21]:
listcol=['description','type','site','secteur','nbemploye','fondation','specialisation']
l=np.eye(len(listurl),len(listcol))
dft2 = pd.DataFrame(l,  columns=listcol)
dft2.head(2)

,description,type,site,secteur,nbemploye,fondation,specialisation
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [22]:
dft2.loc[[1],['description']]

,description
1,0.0


In [24]:
# as usual we use selenium 
for i in range(2609,2030):
    try:
        driver.get(listurl[i])

        button = driver.find_element_by_xpath('/html/body/div[5]/div[4]/div[3]/div/div[2]/div/div[1]/div/div/nav/ul/li[2]/a')
        button.click()
        sleep(2)
        try:
        
            folowers= driver.find_element_by_xpath('/html/body/div[5]/div[4]/div[3]/div/div[2]/div/div[2]/div[1]/section/p')
            dft2.loc[[i],['description']]=folowers.text
            #print("******************")
        except:
            pass
        try:
            type= driver.find_element_by_xpath('/html/body/div[5]/div[4]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[5]')
            dft2.loc[[i],['type']]=type.text
            #print("******************")
        except:
            pass
#fol= driver.find_element_by_xpath('//*[@id="ember64"]/div/div[2]/div[2]')
#followers = fol.text
#upt = secteur.append(followers)
        try:
            site= driver.find_element_by_xpath('/html/body/div[5]/div[4]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[1]/a/span')
            dft2.loc[[i],['site']]=site.text
            #print("******************")
        except:
            pass
        try:
            secteur= driver.find_element_by_xpath('/html/body/div[5]/div[4]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[2]')
            dft2.loc[[i],['secteur']]=secteur.text
            #print("******************")
        except:
            pass
        try:
            nbentreprise= driver.find_element_by_xpath('/html/body/div[5]/div[4]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[3]')
            dft2.loc[[i],['nbemploye']]=nbentreprise.text
            #print("******************")
        except:
            pass
        try:
            fondation= driver.find_element_by_xpath('/html/body/div[5]/div[4]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[6]')
            dft2.loc[[i],['fondation']]=fondation.text
            #print("******************")
        except:
            pass
        try:
            spec= driver.find_element_by_xpath('/html/body/div[5]/div[4]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[7]')
            dft2.loc[[i],['specialisation']]=spec.text
            #print("******************")
        except:
            pass
    except NameError:
        print("Variable x is not defined")
    except:
        print("Something else went wrong")

In [22]:
dft2.head(20)

,description,type,site,secteur,nbemploye,fondation,specialisation
0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0
2,Infor builds business software for specific in...,"New York, NY",http://www.infor.com,Logiciels informatiques,10 001 employés et plus,Société civile/Société commerciale/Autres type...,2002
3,"Créée en 2007, Talan Tunisie est le centre de ...",Société civile/Société commerciale/Autres type...,http://www.talan.tn,Technologies et services de l’information,201-500 employés,2007,ingénierie informatique
4,0,0,0,0,1,0,0
5,0,"Djerba, Medenine",http://www.ido-developers.com/,Technologies et services de l’information,2-10 employés,2018,0
6,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0


- Some Companies haven't a profil and others are private for the scraping 

In [23]:
dft2.to_csv("df_Scrap_final_Entreprise.csv", sep='\t', encoding='utf-8')